In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
from selenium import webdriver

driver = webdriver.Chrome()
driver.get("https://www.imsa.com/weathertech/weathertech-2024-schedule/")

# Wait for dynamic content to load if needed

soup = BeautifulSoup(driver.page_source, 'html.parser')
schedule_details = soup.find_all('div', {'class': 'schedule-details'})

driver.quit()


In [ ]:
# schedule_details

In [3]:
events = []
circuit = []
length = []
dates = []


for div in schedule_details:
    events.append(div.find('h2', class_='schedule-title').text)
    circuit.append(div.find('div', class_='schedule-location').text)
    length.append(div.find('div', class_='schedule-duration').text)
    dates.append(div.find('div', class_='schedule-date').text)

In [4]:
circuit

['Daytona International Speedway',
 'Daytona International Speedway',
 'Sebring International Raceway',
 'Long Beach Street Circuit',
 'WeatherTech Raceway Laguna Seca',
 'Detroit Street Circuit',
 'Watkins Glen International',
 'Canadian Tire Motorsport Park',
 'Road America',
 'VIRginia International Raceway',
 'Indianapolis Motor Speedway',
 'Michelin Raceway Road Atlanta']

In [5]:
length

['DURATION: N/A',
 'DURATION: 24 Hours',
 'DURATION: 12 Hours',
 'DURATION: 100 Minutes',
 'DURATION: 2 Hours 40 Minutes',
 'DURATION: 100 Minutes',
 'DURATION: 6 Hours',
 'DURATION: 2 Hours 40 Minutes',
 'DURATION: 2 Hours 40 Minutes',
 'DURATION: 2 Hours 40 Minutes',
 'DURATION: 6 Hours',
 'DURATION: 10 Hours']

In [6]:
events

['Roar Before The Rolex 24',
 'Rolex 24 At DAYTONA',
 'Mobil 1 Twelve Hours of Sebring',
 'Acura Grand Prix of Long Beach',
 'MOTUL Course de Monterey Powered by Hyundai N',
 'Detroit Grand Prix',
 "Sahlen's Six Hours of The Glen",
 'Chevrolet Grand Prix',
 'IMSA SportsCar Weekend',
 'Michelin GT Challenge At VIR',
 'TireRack.com Battle on the Bricks',
 'MOTUL Petit Le Mans']

In [9]:
dates

['Jan 19 - Jan 21',
 'Jan 24 - Jan 28',
 'Mar 13 - Mar 16',
 'Apr 19 - Apr 20',
 'May 10 - May 12',
 'May 31 - Jun 1',
 'Jun 20 - Jun 23',
 'Jul 12 - Jul 14',
 'Aug 2 - Aug 4',
 'Aug 23 - Aug 25',
 'Sep 20 - Sep 22',
 'Oct 9 - Oct 12']

In [12]:
def convert_dates(dates):
    month_mapping = {
        'Jan': 1,
        'January': 1,
        'Feb': 2,
        'Mar': 3,
        'March': 3,
        'Apr': 4,
        'May': 5,
        'Jun': 6,
        'Jul': 7,
        'Aug': 8,
        'Sep': 9,
        'Oct': 10,
        'October': 10,
        'Nov': 11,
        'Dec': 12
    }

    converted_months = []
    converted_days = []

    for i, date_range in enumerate(dates):
        start_month, start_day, _, end_month, end_day = date_range.split()

        start_month_number = month_mapping[start_month]
        end_month_number = month_mapping[end_month]

        if i == 0:  # Include all days for the roar
            pass
#             for day in range(int(start_day), int(end_day) + 1):
#                 converted_months.append(start_month_number)
#                 converted_days.append(day)
        if i == 1:  # Include both days of 24hr
            for day in range(max(1, int(end_day) - 1), int(end_day) + 1):
                converted_months.append(end_month_number)
                converted_days.append(day)
        else:  # Include only the last day for other events
            converted_months.append(end_month_number)
            converted_days.append(int(end_day))

    return converted_months, converted_days



converted_months, converted_days = convert_dates(dates)

print(converted_months)
print(converted_days)

[1, 1, 1, 3, 4, 5, 6, 6, 7, 8, 8, 9, 10]
[21, 27, 28, 16, 20, 12, 1, 23, 14, 4, 25, 22, 12]


In [ ]:
len(converted_days)


In [ ]:
len(converted_months)

In [ ]:
rounds = []

for i, date_range in enumerate(dates):
    temp_round = f'IMSA WTSC Round {i}'
    rounds.append(temp_round)

rounds.pop(0) 
#pop to remove first event (roar) since it is covered by individual sessions
rounds


In [ ]:
from selenium import webdriver

driver = webdriver.Chrome()
driver.get("https://www.imsa.com/events/2024-rolex-24-at-daytona/")

# Wait for dynamic content to load if needed

soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

In [ ]:
daytona_soup = soup.find('div', {'class':'race-event-schedule-container-inner'})
daytona_soup

In [ ]:
from selenium import webdriver

driver = webdriver.Chrome()
driver.get("https://www.imsa.com/events/2024-roar-before-the-rolex-24/")

# Wait for dynamic content to load if needed

soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

In [ ]:
roar_soup = soup.find('div', {'class':'race-event-schedule-container-inner'})
roar_soup

In [ ]:
schedule_dict = {}
current_header = None

for item in roar_soup.find_all(['div', 'h2']):
    if item.name == 'h2' and item.text == 'Event Schedule':
        continue
    elif item.name == 'div' and 'day-event-header' in item.get('class'):
        current_header = item.text
        schedule_dict[current_header] = []
    elif item.name == 'div' and 'day-event-details-container' in item.get('class') and current_header is not None:
        schedule_dict[current_header].append(item)

In [ ]:
schedule_dict

In [ ]:
# schedule_dict = {}
current_header = None

for item in daytona_soup.find_all(['div', 'h2']):
    if item.name == 'h2' and item.text == 'Event Schedule':
        continue
    elif item.name == 'div' and 'day-event-header' in item.get('class'):
        current_header = item.text
        schedule_dict[current_header] = []
    elif item.name == 'div' and 'day-event-details-container' in item.get('class') and current_header is not None:
        schedule_dict[current_header].append(item)

In [ ]:
schedule_dict
# len(schedule_dict)

In [ ]:
dats = []

for date, session in list(schedule_dict.items())[:-2]:
    for event in events:
#         session_name = session.find('div', class_='event-name').text
#         rounds.insert(0, f"{session_name}")
        dats.append(f"{date}")

In [ ]:
rounds

In [ ]:
# dats
len(dats)

In [ ]:
hh

In [ ]:
dats

In [ ]:
new_dats = []
for i in dats:
    i = i.split(', ', 1)[-1]
    i = i.split()
    new_dats.append(i)
new_dats

In [ ]:
len(new_dats)

In [ ]:
month_mapping = {
    'Jan': 1,
    'January': 1,
    'Feb': 2,
    'Mar': 3,
    'March': 3,
    'Apr': 4,
    'May': 5,
    'Jun': 6,
    'Jul': 7,
    'Aug': 8,
    'Sep': 9,
    'Oct': 10,
    'October': 10,
    'Nov': 11,
    'Dec': 12
}

conv_months = []
conv_days = []

for date_list in new_dats:
    month_name, day, year = date_list
    conv_months.append(month_mapping[month_name])
    conv_days.append(int(day.strip(',')))

conv_months.sort()


In [ ]:
# conv_months
len(conv_months)

In [ ]:
for i in conv_months:
    converted_months.insert(0, i)


In [ ]:
# converted_months
len(converted_months)

In [ ]:
for i in reversed(conv_days):
    converted_days.insert(0, i)

In [ ]:
# converted_days
len(converted_days)

In [ ]:
rounds

In [ ]:
# rounds = ['some text', 'some more test', 'other text']

for date, events in reversed(list(schedule_dict.items())[:-2]): #drops the last two items since they are the same as 'round 1'
    for event in reversed(events):
        event_name = event.find('div', class_='event-name').text
        rounds.insert(0, f"{event_name}")

In [ ]:
rounds
# len(rounds)

In [ ]:
circuit


In [ ]:
for i in enumerate(new_dats):
    circuit.insert(0, 'Daytona International Speedway')
    length.insert(0, 'DURATION: N/A')
    events.insert(0, 'test')

In [ ]:
circuit

In [ ]:
length

In [ ]:
events

In [ ]:
import pytz
import icalendar
from datetime import datetime
from pathlib import Path
import os
from icalendar import Calendar, Event,  vText, vCalAddress

In [ ]:
IMSA_WTSC = Calendar()
IMSA_WTSC.add('prodid', '-//My calendar product//example.com//')
IMSA_WTSC.add('version', '2.0')

In [ ]:
uid = 110

for round, circuit, day, month, length, event in zip(rounds, circuit, converted_days, converted_months, length, events):
    ievent = Event()
    ievent.add('summary', round) #Title of the event
#     ievent.add('description', event)     #Description of event
#     ievent.add('description', f"{event}\n{length}")

  
    # Format the length for a cleaner display
    formatted_length = length.replace('DURATION: ', '')
    
    # Include the formatted length in the description
    ievent.add('description', f"{event}\nDuration: {formatted_length}")

    ievent.add('dtstart', datetime(2024, month, day, 14, 0, 0, tzinfo=pytz.timezone('est')))
    ievent.add('dtend', datetime(2024, month, day, 16, 40, 0, tzinfo=pytz.timezone('est')))
    ievent.add('dtstamp', datetime(2024, 1, 9, 0, 10, 0, tzinfo=pytz.timezone('est')))
    ievent.add('location', vText(circuit))
    ievent.add('uid', uid)
    uid += 1
    IMSA_WTSC.add_component(ievent)

In [ ]:
IMSA_WTSC

In [ ]:
directory = str(Path(r'C:\Users\chris\Documents\Calendars')) + "/"
print("ics file will be generated at ", directory)
f = open(os.path.join(directory, 'IMSA_WTSC_2024.ics'), 'wb')
f.write(IMSA_WTSC.to_ical())
f.close()